Heston Model

dS_t = mu * S_t * dt + sqrt(V_t) * S_t * dW_t_1

dV_t = k*(theta - V_t) + sigma * sqrt(V_t) * dW_t_2

S_t : value of asst at time t  
mu: expected growth rate of the log normal stock value  
V_t : the variance of the asset S_t  
W_t_1 : the stochastic process governing the S_t process  
theta: value of mean reversion for the variance V_t  
k: the strength of mean reversion   
sigma: volatility of volatility  
W_t_2: the stochastic process governing the V_t process  
rho : correlation between W_t_1 and W_t_2

In [1]:
from QuantLib import *
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import math

Europearn call option for AAPL  
strike price: 130  
Maturity: 15th Jan 2016  
spot price: 127.62  
volatility of stock: 20%  
dividend yield: 1.62%  
short time risk free rate: 0.1%  
current data: 8th, May, 2015

In [4]:
# option parameter
# strike_price = 110.0
# payoff = PlainVanillaPayoff(Option.Call, Strike_price)

# option data
maturity_date = Date(15, 1, 2016)
spot_price = 127.62
strike_price = 130.0
volatility = 0.20
dividend_rate = 0.0163
option_type = Option.Call

risk_free_rate = 0.001
day_count = Actual365Fixed()
calendar = UnitedStates()

calculation_date = Date(8, 5, 2015)
Settings.instance().evaluationDate = calculation_date

In [8]:
# construct the European Option

payoff = PlainVanillaPayoff(option_type, strike_price)
exercise = EuropeanExercise(maturity_date)
european_option = VanillaOption(payoff, exercise)

In [9]:
# construct the Heston Process

v0 = volatility * volatility
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

spot_handle = QuoteHandle(SimpleQuote(spot_price))
flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
heston_process = HestonProcess(flat_ts, dividend_yield, spot_handle, v0, kappa, theta, sigma, rho)

In [16]:
engine = AnalyticHestonEngine(HestonModel(heston_process), 0.01, 1000)
european_option.setPricingEngine(engine)
h_price = european_option.NPV()
print ("The Heston Model price is ", h_price)

The Heston Model price is  6.533866879681


In [17]:
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, volatility, day_count))
bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))

bs_price = european_option.NPV()
print("The Black-Scholes-Merton model price is ", bs_price)

The Black-Scholes-Merton model price is  6.749271812460607


The difference is due to the stochastic modeling of the volatility as a CIR-process